In [7]:
import os 
import re 
import cantera as ct

directory ='../'
species = {'CH2F2': [0.11,0.225,0.174] ,'CH3F': [0.08,.225,0.123],'C2H5F': [0.025,0.150,0.0654],'CH2FCH2F' :[0.0375,0.150,0.0775],'CH2FCHF2' :[0.075,0.150,0.095],'CH3CF3' :[0.07,0.150,0.095],'CH3CHF2':[0.0375, 0.150,0.0775]}

 

In [8]:
def make_vol_frac_list():

    block = \
   '''
To = 298
Po = ct.one_atm

vol_frac_list = list(np.linspace(0.025, 0.25, 30))


gas = ct.Solution(f'./cantera/chem.cti')
results = {}  

  
#this part calculates the flame calcs for all of the vol_frac list, using the previous vol frac count as a guess for the current one
for i in  range(len(vol_frac_list)):
    try:
        
        tol_ss = [1.0e-13, 1.0e-9]  #abs and rel tolerances for steady state problem
        tol_ts = [1.0e-13, 1.0e-9]  #abs and rel tie tolernces for time step function

        print(f'****************************starting new volume fraction: {x}**************************')
        
        x = vol_frac_list[i]
        norm_ox = (1-x)*.21

'''
    
    return block

In [9]:
def make_vol_frac_dict(label_list,num):
    '''Makes a volume fraction dictionary, assuming there are equal parts fuel in the blend'''

    vol_frac_dict = \
    '''
        vol_frac_dict = {'''
    l = list(range(len(label_list)-1))
    for i in l:
        vol_frac_dict += f"'{label_list[i]}': (x/{num}/norm_ox), "
    vol_frac_dict += f"'{label_list[-1]}':((1-x)*.21)/norm_ox, 'N2':((1-x)*0.79)/norm_ox"
    vol_frac_dict += '}'
    
    block2 = \
    '''
    
        print(vol_frac_dict)
        print(f"O2/CH4 ratio = {vol_frac_dict['O2(4)']/vol_frac_dict['CH4(3)']}. Complete combustion takes 2")
        
    

        gas.TPX = To, Po, vol_frac_dict
        width = 0.08
        flame = ct.FreeFlame(gas, width=width)
        flame.flame.set_steady_tolerances(default=tol_ss)   #set tolerances
        flame.flame.set_transient_tolerances(default=tol_ts)
        flame.set_refine_criteria(ratio=5, slope=0.25, curve=0.27)
        #flame.set_refine_criteria(ratio=3, slope=0.1, curve=0.1) 
        flame.max_time_step_count = 900
        loglevel = 1 
        if i!=0:
            d = f'./data/volume_fractions/test_{vol_frac_list[i-1]}.csv'
            if os.path.exists(d):  
                arr2 = ct.SolutionArray(gas)
                arr2.read_csv(d)
                flame.set_initial_guess(data=arr2)
                print(' initial guess has been set')
        #"False" stops the calculation from retrying over and over, thanks Chao 
        flame.solve(loglevel=loglevel, auto=False)
        #flame.solve(loglevel=loglevel, auto=True)
        Su = flame.velocity[0]
        results[x] = Su
        eq_ratios.append(eq_ratio)
        sltn = flame.to_solution_array()
        df1 = sltn.to_pandas()
        df1.to_csv(f'./data/volume_fractions/test_{vol_frac_list[i-1]}.csv', index=False)            
            
    except Exception as e: 
        print(f'********************passed volume fraction:{vol_frac_list[i]}, error: {e}*************************************')
        pass

vol_fracs = list(results.keys())
flame_speeds = list(results.values())


print("volume fractions are:")
print(vol_fracs)

print("flame speeds are:")
print(flame_speeds)

    
with open('final_calcs.csv', 'w+') as g:
    writers = csv.writer(g)
    writers.writerow(vol_fracs)
    writers.writerow(flame_speeds)
    
        
'''
    vol_frac_dict += block2
    return vol_frac_dict

In [10]:
def make_bash_script(label_list):
    bash_script = f'''#!/bin/sh

#SBATCH --nodes=1
#SBATCH --time=7-00:00:00
#SBATCH --job-name=fc_{'_'.join(label_list[0:-1])}
#SBATCH --error=slurm/error/fc_error.slurm.log
#SBATCH --output=slurm/output/fc_output.slurm.log
##SBATCH --cpus-per-task=5
##SBATCH --mem-per-cpu=8Gb
##SBATCH --ntasks=1 
##SBATCH --array=1
#SBATCH --partition=west

source activate cantera_env
python flame_speed_calc.py

'''
    return bash_script

In [11]:
def make_blends(list_of_blend_files, num):
    """Provided with a list of blend files, 
    specify the number of fuel species in the blend (num), 
    this will run flame speed calc on all of them.
    
    example: make_blend([list using os.listdir(directory)], num)
    """
    
    
    flame_speed_calc = '''
    
#########calculates flamespeeds for the .cti files at different equivalence ratios. Uses initial guess from the previous model #############################


import cantera as ct
import numpy as np
import pandas as pd
import os
import csv 

print("Running Cantera Version: " + str(ct.__version__))

    '''

    for file in list_of_blend_files: 
  
        #make pattern that re.match will use. Ex) for blends of 2 fuels (C2H5F, CH3F), pattern will be 'C2H5F_CH3F'
        pattern = ''
        l = list(range(num-1))
        for i in l:
            pattern += '([A-Z0-9]+)_'
        pattern += '([A-Z0-9]+)'
        

        #match and make a list of the species matched in each file
        match = re.match(pattern, file)
        label = []
        l = list(range(num))
        for i in l: 
            label.append(match.group(i+1))
     
        #correct label if it has a funky name. Will use this label list to get RMG index of species name. 
        #Ex) the file name uses 'CH2FCHF2', but RMG says this species is 'C2H3F3'
        labels_we_hate={'CH2FCHF2':'C2H3F3','CH2FCH2F':'C2H4F2'}
        for i in range(len(label)):
            if label[i] in labels_we_hate.keys(): 
                label[i]= labels_we_hate[label[i]]
        #print(label)
 
        #add oxygen to list, and rename 
        ox_label = 'O2' 
        species_to_look_for = [i for i in label]
        species_to_look_for.append(ox_label)
        #print(species_to_look_for)

        #get species name from cti file
        chem_file = f'../{file}/cantera/chem.cti'
        gas = ct.Solution(chem_file)
        species_names_we_want = {}
        for i in species_to_look_for:
            for name in gas.species_names:
                print(name)
                pattern = f"{i}\([0-9]+\)"
                m = re.match(pattern, name)
                if m:
                    species_names_we_want[i]=name
                    #print(i, name)
                    break
                else:
                    print(f"still looking for {i}")
        print(species_names_we_want)
        print('Does the dictionary look ok?')
                    


        
        #now make the damn thing
        flame_speed_calc += make_vol_frac_list()
        flame_speed_calc += make_vol_frac_dict(list(species_names_we_want.values()),num)
        bash_script = make_bash_script(list(species_names_we_want.values()))
        d = os.path.join(directory,file)
        os.makedirs(d,exist_ok=True)
        slurm_error = os.path.join(d,'slurm/error')
        slurm_output = os.path.join(d, 'slurm/output')
        os.makedirs(slurm_error,exist_ok=True)
        os.makedirs(slurm_output,exist_ok=True)
        with open(os.path.join(d,f'flame_speed_calc.py'),'w') as f: 
            for l in flame_speed_calc: 
                f.write(l)
        with open(os.path.join(d,f'flame_speed_run.sh'),'w') as f: 
            for l in bash_script: 
                f.write(l)
                
 
  



In [12]:

list_of_blends = [file for file in os.listdir(directory) if re.match('[A-Z0-9]+_[A-Z0-9]+', file)]
#list_of_blends = ['C2H5F_CH2F2']

make_blends(list_of_blends, 2)

N2
still looking for C2H5F
Ne
still looking for C2H5F
C2H5F(1)
N2
still looking for CH2F2
Ne
still looking for CH2F2
C2H5F(1)
still looking for CH2F2
CH2F2(2)
N2
still looking for O2
Ne
still looking for O2
C2H5F(1)
still looking for O2
CH2F2(2)
still looking for O2
O2(3)
{'C2H5F': 'C2H5F(1)', 'CH2F2': 'CH2F2(2)', 'O2': 'O2(3)'}
Does the dictionary look ok?
N2
still looking for C2H5F
Ne
still looking for C2H5F
C2H5F(1)
N2
still looking for C2H4F2
Ne
still looking for C2H4F2
C2H5F(1)
still looking for C2H4F2
C2H4F2(2)
N2
still looking for O2
Ne
still looking for O2
C2H5F(1)
still looking for O2
C2H4F2(2)
still looking for O2
O2(3)
{'C2H5F': 'C2H5F(1)', 'C2H4F2': 'C2H4F2(2)', 'O2': 'O2(3)'}
Does the dictionary look ok?
N2
still looking for C2H5F
Ne
still looking for C2H5F
C2H5F(1)
N2
still looking for C2H3F3
Ne
still looking for C2H3F3
C2H5F(1)
still looking for C2H3F3
C2H3F3(2)
N2
still looking for O2
Ne
still looking for O2
C2H5F(1)
still looking for O2
C2H3F3(2)
still looking for O2
O

In [21]:
eq_ratios = []
vol_frac_list = [.095]
for i in vol_frac_list:
    x = i
    norm_ox = (1-x)*.21
    vol_frac_dict = {'CH4(3)': (x/2/norm_ox), 'hi': (x/2/norm_ox), 'O2(4)':((1-x)*.21)/norm_ox, 'N2':((1-x)*0.79)/norm_ox}
    F_A = (vol_frac_dict['CH4(3)']*2)/vol_frac_dict['O2(4)']
    F_A_stoic = 0.5
    eq_ratio = F_A/F_A_stoic
    
    
    
    
    
    eq_ratios.append(eq_ratio)

print(eq_ratio)
print(eq_ratios)


0.9997369113391213
[0.9997369113391213]


In [24]:
import os
import re
direc = '/work/westgroup/nora/Code/projects/halogens/refrigerants/blends'
list_of_blends = [file for file in os.listdir(direc) if re.match('[A-Z0-9]+_[A-Z0-9]+', file)]
for file in list_of_blends: 
    command = f'scp /work/westgroup/nora/Code/projects/halogens/refrigerants/blends/{file}/chemkin/dups/dup_chem.cti /work/westgroup/nora/Code/projects/halogens/refrigerants/patterns_in_blends/blends_of_2/{file}/cantera/chem.cti'
    os.system(command)

In [23]:
import os
import re
direc = '/work/westgroup/nora/Code/projects/halogens/refrigerants/blends'
list_of_blends = [file for file in os.listdir(direc) if re.match('[A-Z0-9]+_[A-Z0-9]+', file)]
for file in list_of_blends: 
    command = f'rm /work/westgroup/nora/Code/projects/halogens/refrigerants/patterns_in_blends/blends_of_2/{file}/cantera/chem.cti'
    os.system(command)